In [1]:
from importlib import reload
from utilities import *
import agent
reload(agent)

2024-04-20 06:58:13.957883: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


<module 'agent' from '/Users/nagajas/Desktop/rl_trader_AI/agent.py'>

In [2]:
stocks = ['AAPL', 'GOOGL', 'AMZN', 'ADBE']

In [3]:
episode_count = 10
true, diff = allStates(1000)
split = int(0.9*len(true[0]))

true_train, true_test = true[:split], true[:split]
diff_train, diff_test = diff[:split], diff[:split]

l = len(diff_train[0]) - 1

Model found for  AAPL
Model found for  GOOGL
Model found for  AMZN
Model found for  ADBE
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step


In [5]:
p_agent = agent.DQNAgent(4)

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 4, 1)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 4, 64)          │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 4, 32)          │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 4, 3)           │            99 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,307 (9.01 KB)

 Trainable params: 2,307 (9.01 KB)

 Non-trainable params: 0 (0.00 B)

### $\textbf{Training Agent}$

In [6]:
for e in range(1,episode_count + 1):
    
    print("Episode " + str(e) + "/" + str(episode_count))
    state = getstate(diff_train, 0)
    total_profit = 0
    p_agent.inventory = OrderedDict()
    p_agent.replay_buffer.clear()
    for stock in stocks:
        p_agent.inventory[stock] = []

    for t in tqdm(range(l)):
        actions = p_agent.epsilon_greedy(state)
        # sit
        next_state = getstate(diff_train, t+1)
        reward = np.zeros((4,1))
        
        for i in range(len(actions)):
            action = actions[i]
            stock = stocks[i]
            
            if action == 1: # buy
                p_agent.inventory[stock].append(true_train[i,t])
                
            elif action == 2 and len(p_agent.inventory[stock]) > 0: # sell
                bought_price = p_agent.inventory[stock].pop(0)
                reward[i] = np.max(true_train[i,t] - bought_price, 0)
                
                total_profit += true_train[i,t] - bought_price
        # print(reward)
        done = True if t == l - 1 else False
        p_agent.store_transition(state, actions, reward, next_state, done)
        state = next_state

        if len(p_agent.replay_buffer) > 32:
            p_agent.exp_replay(32)
            p_agent.replay_buffer.clear()        
        # print("t:::" , t)

Episode 1/10


100%|██████████| 926/926 [03:15<00:00,  4.74it/s]


Episode 2/10


100%|██████████| 926/926 [03:12<00:00,  4.81it/s]


Episode 3/10


...

100%|██████████| 926/926 [03:39<00:00,  4.22it/s]


Episode 9/10


100%|██████████| 926/926 [04:18<00:00,  3.58it/s]


Episode 10/10


100%|██████████| 926/926 [04:20<00:00,  3.55it/s]


### $\textbf{Evaluating Agent}$

In [9]:
state = getstate(diff_test, 0)
total_profit = 0
p_agent.inventory = OrderedDict()

for stock in stocks:
    p_agent.inventory[stock] = []
l = len(diff_test[0])
for t in range(len(diff_test[0])-1):
    actions = p_agent.epsilon_greedy(state, eval = True)
    # sit
    next_state = getstate(diff_test, t+1)
    reward = np.zeros((4,1))
    
    print("Day: ", t+1)
    for i in range(len(actions)):
        action = actions[i]
        stock = stocks[i]
        
        if action == 0 and len(p_agent.inventory[stock]) > 0:
            print("Stock: ", stock, "Action: Hold: $", true_test[i,t])
        
        elif action == 1: # buy
            p_agent.inventory[stock].append(true_test[i,t])
            print("Stock: ", stock, "Action: Buy: $", true_test[i,t])
            
        elif action == 2 and len(p_agent.inventory[stock]) > 0: # sell
            bought_price = p_agent.inventory[stock].pop(0)
            reward[i] = np.max(true_test[i,t] - bought_price, 0)
            # print(reward)
            total_profit += true_test[i,t] - bought_price
            print("Stock: ",stock,"Action: Sell: $", (true_test[i,t]),)

    done = True if t == l - 2 else False
    p_agent.store_transition(state, actions, reward, next_state, done)
    state = next_state
    if done:
        print("-----------------------")
        print(" Total Profit: ${:.2f} ".format(total_profit))
        print("-----------------------")

Day:  1
Stock:  AAPL Action: Buy: $ 90.796829
Stock:  GOOGL Action: Buy: $ 75.837502
Stock:  ADBE Action: Buy: $ 431.73999
Day:  2
Stock:  AAPL Action: Sell: $ 90.571907
Stock:  GOOGL Action: Buy: $ 75.4105
Stock:  AMZN Action: Buy: $ 150.445496
Stock:  ADBE Action: Buy: $ 430.309998
Day:  3
Stock:  AAPL Action: Buy: $ 92.718452
Stock:  GOOGL Action: Sell: $ 76.471497
Stock:  AMZN Action: Sell: $ 152.760498
Stock:  ADBE Action: Sell: $ 437.100006
Day:  4
Stock:  AAPL Action: Sell: $ 91.195351
Stock:  GOOGL Action: Buy: $ 75.182503
Stock:  AMZN Action: Buy: $ 150.016495
Stock:  ADBE Action: Buy: $ 432.26001
Day:  5
Stock:  AAPL Action: Buy: $ 92.94339
Stock:  GOOGL Action: Sell: $ 76.175499
Stock:  AMZN Action: Sell: $ 151.676498
Stock:  ADBE Action: Sell: $ 436.299988
Day:  6
Stock:  AAPL Action: Sell: $ 94.068016
Stock:  GOOGL Action: Sell: $ 76.918503
Stock:  AMZN Action: Buy: $ 152.593994
Stock:  ADBE Action: Sell: $ 438.880005
Day:  7
Stock:  AAPL Action: Buy: $ 103.915894
Stock:  